In [ ]:
import torchvision.models as models

In [ ]:
resnet18 = models.resnet18(pretrained=True)

In [ ]:
resnet18.eval()

In [ ]:
#del features
features = []

def hook(module, input, output):
    features.append(output)

# Assuming you want to extract features after each of the four main blocks
layers = [
    resnet18.layer1,
    resnet18.layer1[0],
    resnet18.layer2,
    resnet18.layer2[0],
    resnet18.layer3,
    resnet18.layer3[0],
    resnet18.layer4,
    resnet18.layer4[0],
    resnet18.fc
]

for layer in layers:
    layer.register_forward_hook(hook)

In [ ]:
layers

In [ ]:
import torch
final_layer1_b1 = torch.empty((0, 64, 56, 56))
final_layer2_b1 = torch.empty((0, 128, 28, 28))
final_layer3_b1 = torch.empty((0, 256, 14, 14))
final_layer4_b1 = torch.empty((0, 512, 7, 7))

final_layer1_b2 = torch.empty((0, 64, 56, 56))
final_layer2_b2 = torch.empty((0, 128, 28, 28))
final_layer3_b2 = torch.empty((0, 256, 14, 14))
final_layer4_b2 = torch.empty((0, 512, 7, 7))

final_fc = torch.empty((0,1000))

In [ ]:
feature_vals = {}

In [ ]:
import opendatasets as od

In [ ]:
!mv ./kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import kaggle
import opendatasets as od

In [ ]:
!kaggle competitions download -c imagenet-object-localization-challenge

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision import transforms
import os
from PIL import Image

class FlatDirectoryImages(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.images = [f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        # Using PIL Image to ensure compatibility with torchvision transforms
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image

# Define your transforms
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Instantiate the dataset
dataset = FlatDirectoryImages(root_dir='data/ILSVRC/Data/CLS-LOC/val', transform=transform)

In [ ]:
dataset.__len__()

In [ ]:
# Create a DataLoader
data_loader = DataLoader(dataset, batch_size=256, shuffle=False, num_workers=8)

In [ ]:
resnet18.to('cuda')
features=[]
resnet18.eval()
# Don't forget to move your input to the same device as your model

with torch.no_grad():
    for i,images in enumerate(data_loader):
        del features
        features=[]
        images = images.to('cuda')
        print(i)
        outputs = resnet18(images)
        print(len(features))
        final_layer1_b1 = torch.cat((final_layer1_b1,features[0].cpu()), dim=0)
        final_layer2_b1 = torch.cat((final_layer2_b1,features[2].cpu()), dim=0)
        final_layer3_b1 = torch.cat((final_layer3_b1,features[4].cpu()), dim=0)
        final_layer4_b1 = torch.cat((final_layer4_b1,features[6].cpu()), dim=0)
        
        final_layer1_b2 = torch.cat((final_layer1_b2,features[1].cpu()), dim=0)
        final_layer2_b2 = torch.cat((final_layer2_b2,features[3].cpu()), dim=0)
        final_layer3_b2 = torch.cat((final_layer3_b2,features[5].cpu()), dim=0)
        final_layer4_b2 = torch.cat((final_layer4_b2,features[7].cpu()), dim=0)
        
        final_fc = torch.cat((final_fc,features[8].cpu()), dim=0)
        torch.cuda.empty_cache()
        features.clear()
        # if i==20:
        #     break
        #torch.cuda.empty_cache()
        #del images
        # Your code to handle the outputs goes here

In [ ]:
print(final_layer1_b1.shape)
print(final_layer2_b1.shape)
print(final_layer3_b1.shape)
print(final_layer4_b1.shape)
print(final_layer1_b2.shape)
print(final_layer2_b2.shape)
print(final_layer3_b2.shape)
print(final_layer4_b2.shape)
print(final_fc.shape)

In [ ]:
torch.save(final_layer1_b1,'output/final_layer1_b1.pt')
torch.save(final_layer2_b1,'output/final_layer2_b1.pt')
torch.save(final_layer3_b1,'output/final_layer3_b1.pt')
torch.save(final_layer4_b1,'output/final_layer4_b1.pt')
torch.save(final_layer1_b2,'output/final_layer1_b2.pt')
torch.save(final_layer2_b2,'output/final_layer2_b2.pt')
torch.save(final_layer3_b2,'output/final_layer3_b2.pt')
torch.save(final_layer4_b2,'output/final_layer4_b2.pt')
torch.save(final_fc,'output/final_fc.pt')